# Historical averages grouped by flight number, to replace op_carrier_fl_num feature. 

In [18]:
import sys
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file

from custom_scripts import database
from custom_scripts import preprocessing
from custom_scripts import preparation
from custom_scripts import modeling
import pandas as pd
import numpy as np

In [19]:
raw_test_flights = preprocessing.get_test_flights()
average_delays = pd.read_csv('../../data/preprocessing/averages_by_fl_num.csv')



In [20]:
test_flights = raw_test_flights.copy()
print(test_flights.shape)
test_flights.head(1)

(150623, 9)


,fl_date,op_unique_carrier,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01,WN,5888,13891,14771,1810,1945,95,363


In [21]:
### Historic average delay features
test_flights = pd.merge(test_flights, average_delays, on='op_carrier_fl_num')
test_flights.head(1)

,fl_date,op_unique_carrier,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,fl_num_avg_dep_delay,...,fl_num_crs_elapsed_time,fl_num_actual_elapsed_time,fl_num_air_time,fl_num_carrier_delay,fl_num_weather_delay,fl_num_avg_nas_delay,fl_num_security_delay,fl_num_late_aircraft_delay,fl_num_total_add_gtime,fl_num_longest_add_gtime
0,2020-01-01,WN,5888,13891,14771,1810,1945,95,363,12.661189,...,111.951254,108.02666,80.200679,24.014925,1.831557,14.498934,0.083156,30.010661,47.090909,47.090909
